## Importing the relevant packages

In [79]:
import numpy as np 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Outling the model

In [80]:
input_size = 784
output_size = 10
hidden_layer_size = 100

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

outputs = tf.matmul(outputs_2, weights_3) + biases_3

### Adding hidden layers

#weights_4 = tf.get_variable('weights_4', [hidden_layer_size, output_size])
#biases_4 = tf.get_variable("biases_4", [output_size])

#outputs = tf.matmul(outputs_3, weights_4) +  biases_4


#### Loss and Optmization Algorithm

In [81]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

#### Calculating the Accuracy of the Model

In [82]:
# Predicting the accuracy of the cases did the algorithm assign the highest 
# probability to the output that matched the target

out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))  

accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

#### Batching and Early stopping

In [83]:
sess = tf.InteractiveSession()

initilizer = tf.global_variables_initializer()

sess.run(initilizer)

batch_size = 100

batches_number = mnist.train._num_examples // batch_size

max_epochs = 15

prev_validation_loss = 99999999.

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0. 

    for batch_counter in range(batches_number):
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        _, batch_loss = sess.run([optimize, mean_loss],
                                 feed_dict= {inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss

    curr_epoch_loss /= batches_number

    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                                    feed_dict = {inputs: input_batch, targets: target_batch})
    

    print('Epoch '+str(epoch_counter + 1) + 
          '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss) + 
          '.Validation loss: '+'{0:.3f}'.format(validation_loss) + 
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.) + '%')
    
    if validation_loss > prev_validation_loss:
        break

    prev_validation_loss = validation_loss

print('End of training.')

c:\Users\assis\anaconda3\envs\py36\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1. Training loss: 0.342.Validation loss: 0.174. Validation accuracy: 94.96%
Epoch 2. Training loss: 0.141.Validation loss: 0.107. Validation accuracy: 96.96%
Epoch 3. Training loss: 0.097.Validation loss: 0.102. Validation accuracy: 97.14%
Epoch 4. Training loss: 0.076.Validation loss: 0.094. Validation accuracy: 97.26%
Epoch 5. Training loss: 0.060.Validation loss: 0.080. Validation accuracy: 97.80%
Epoch 6. Training loss: 0.048.Validation loss: 0.071. Validation accuracy: 97.74%
Epoch 7. Training loss: 0.042.Validation loss: 0.074. Validation accuracy: 97.72%
End of training.


We don't train on the validation dataset, but we adjust the model (hyperparameters)
considering the results from the validation

### Test

In [84]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy],
                         feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0] * 100. 

print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent))

# The test accuracy is the final accuracy of the model

Test accuracy: 97.57
